# Scenario 00: Environment Setup

**Estimated Time**: 15 minutes

## Overview
This notebook verifies your environment is correctly set up for the Agentic AI Patterns Workshop.

## What We'll Check
- Python version (3.11+)
- Required packages installed
- Environment variables configured
- Azure OpenAI connectivity
- Telemetry setup

## 1. Python Version Check

In [ ]:
import sys

print(f"Python version: {sys.version}")

version_info = sys.version_info
if version_info >= (3, 11):
    print("✅ Python version is 3.11 or higher")
else:
    print(f"❌ Python {version_info.major}.{version_info.minor} detected. Python 3.11+ required.")

## 2. Required Packages Check

In [ ]:
import importlib
from typing import NamedTuple


class PackageCheck(NamedTuple):
    name: str
    import_name: str
    required: bool = True


REQUIRED_PACKAGES = [
    PackageCheck("pydantic", "pydantic"),
    PackageCheck("pydantic-settings", "pydantic_settings"),
    PackageCheck("opentelemetry-api", "opentelemetry"),
    PackageCheck("opentelemetry-sdk", "opentelemetry.sdk"),
    PackageCheck("openai", "openai"),
    PackageCheck("fastapi", "fastapi"),
    PackageCheck("httpx", "httpx"),
    PackageCheck("mcp", "mcp", required=False),  # May not be available yet
    PackageCheck("python-dotenv", "dotenv"),
]

print("Checking required packages...\n")

all_passed = True
for pkg in REQUIRED_PACKAGES:
    try:
        module = importlib.import_module(pkg.import_name)
        version = getattr(module, "__version__", "unknown")
        print(f"✅ {pkg.name}: {version}")
    except ImportError:
        if pkg.required:
            print(f"❌ {pkg.name}: NOT INSTALLED")
            all_passed = False
        else:
            print(f"⚠️ {pkg.name}: NOT INSTALLED (optional)")

print()
if all_passed:
    print("✅ All required packages are installed")
else:
    print("❌ Some required packages are missing. Run: pip install -e .")

In [ ]:
from opentelemetry.instrumentation.logging import LoggingInstrumentor

## 3. Configuration Loading

In [ ]:
# Load environment variables from .env file
from dotenv import load_dotenv

# Look for .env in parent directory (project root)
from pathlib import Path

env_path = Path("../.env")
if env_path.exists():
    load_dotenv(env_path)
    print(f"✅ Loaded .env from {env_path.resolve()}")
else:
    print(f"⚠️ No .env file found at {env_path.resolve()}")
    print("   Copy .env.example to .env and configure your settings.")

In [ ]:
# Add src to path for imports
import sys
from pathlib import Path

project_root = Path("..").resolve()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
    print(f"✅ Added {project_root} to Python path")

In [ ]:
from src.common.config import get_settings

# Clear cache to reload with new env vars
get_settings.cache_clear()

try:
    settings = get_settings()
    print("✅ Configuration loaded successfully\n")
    
    # Display configuration status (hide sensitive values)
    print("Configuration Status:")
    print(f"  Azure OpenAI Endpoint: {'✅ Set' if settings.azure_openai_endpoint else '❌ Not set'}")
    print(f"  Azure OpenAI API Key: {'✅ Set' if settings.azure_openai_api_key else '⚠️ Not set (will use DefaultAzureCredential)'}")
    print(f"  Azure OpenAI Deployment: {settings.azure_openai_deployment or 'Not set'}")
    print(f"  API Version: {settings.azure_openai_api_version}")
    print(f"  Application Insights: {'✅ Set' if settings.applicationinsights_connection_string else '⚠️ Not set'}")
    print(f"  OpenTelemetry Enabled: {settings.enable_otel}")
    print(f"  Log Level: {settings.log_level}")
    print()
    print(f"  Azure Configured: {'✅ Yes' if settings.is_azure_configured else '❌ No'}")
    print(f"  Observability Configured: {'✅ Yes' if settings.is_observability_configured else '⚠️ No (using console exporter)'}")

except Exception as e:
    print(f"❌ Configuration error: {e}")
    print("\nPlease check your .env file settings.")

## 4. Telemetry Setup

In [ ]:
from src.common.telemetry import setup_telemetry, get_tracer, is_telemetry_enabled

try:
    setup_telemetry()
    print(f"✅ Telemetry initialized: {is_telemetry_enabled()}")
    
    # Test tracer creation
    tracer = get_tracer(__name__)
    print(f"✅ Tracer created successfully")
    
    # Create a test span
    with tracer.start_as_current_span("setup_verification") as span:
        span.set_attribute("notebook", "00_setup")
        span.set_attribute("status", "success")
        print("✅ Test span created")

except Exception as e:
    print(f"❌ Telemetry setup error: {e}")

## 5. Azure OpenAI Connectivity (Optional)

In [ ]:
from src.common.config import get_settings

settings = get_settings()

if not settings.is_azure_configured:
    print("⚠️ Azure OpenAI is not configured. Skipping connectivity test.")
    print("   Set AZURE_OPENAI_ENDPOINT in your .env file to enable.")
else:
    print("Azure OpenAI is configured. Testing connectivity...")
    
    try:
        from openai import AzureOpenAI
        
        # Create client
        if settings.azure_openai_api_key:
            client = AzureOpenAI(
                azure_endpoint=settings.azure_openai_endpoint,
                api_key=settings.azure_openai_api_key,
                api_version=settings.azure_openai_api_version,
            )
        else:
            from azure.identity import DefaultAzureCredential, get_bearer_token_provider
            
            token_provider = get_bearer_token_provider(
                DefaultAzureCredential(),
                "https://cognitiveservices.azure.com/.default"
            )
            client = AzureOpenAI(
                azure_endpoint=settings.azure_openai_endpoint,
                azure_ad_token_provider=token_provider,
                api_version=settings.azure_openai_api_version,
            )
        
        # Simple test call
        if settings.azure_openai_deployment:
            response = client.chat.completions.create(
                model=settings.azure_openai_deployment,
                messages=[{"role": "user", "content": "Say 'Hello, Workshop!' in 5 words or less."}],
                max_tokens=20,
            )
            print(f"✅ Azure OpenAI connectivity test passed")
            print(f"   Response: {response.choices[0].message.content}")
            print(f"   Model: {response.model}")
            print(f"   Tokens used: {response.usage.total_tokens}")
        else:
            print("⚠️ AZURE_OPENAI_DEPLOYMENT not set. Cannot test model.")
            
    except Exception as e:
        print(f"❌ Azure OpenAI connectivity test failed: {e}")
        print("\nPlease verify:")
        print("  - Your Azure OpenAI endpoint is correct")
        print("  - Your API key or Azure credentials are valid")
        print("  - The deployment name exists in your Azure OpenAI resource")

## 6. Exception Handling Test

In [ ]:
from src.common.exceptions import (
    WorkshopError,
    ConfigurationError,
    AgentError,
    ToolError,
    MCPError,
    WorkflowError,
)

print("Testing custom exceptions...\n")

# Test each exception type
exceptions_to_test = [
    ("WorkshopError", WorkshopError("Base error", details={"test": True})),
    ("ConfigurationError", ConfigurationError("Config missing", config_key="API_KEY")),
    ("AgentError", AgentError("Agent failed", agent_name="test_agent")),
    ("ToolError", ToolError("Tool error", tool_name="search")),
    ("MCPError", MCPError("Connection failed", server_url="http://localhost:8001")),
    ("WorkflowError", WorkflowError("Step failed", workflow_name="pipeline", step_name="analyze")),
]

all_passed = True
for name, exc in exceptions_to_test:
    try:
        raise exc
    except WorkshopError as e:
        print(f"✅ {name}: {e}")
    except Exception as e:
        print(f"❌ {name}: Unexpected exception type: {type(e)}")
        all_passed = False

print()
if all_passed:
    print("✅ All exception types working correctly")
else:
    print("❌ Some exceptions failed")

## Summary

In [ ]:
import sys
from src.common.config import get_settings
from src.common.telemetry import is_telemetry_enabled

settings = get_settings()

print("="*50)
print("WORKSHOP SETUP SUMMARY")
print("="*50)
print()

checks = [
    ("Python 3.11+", sys.version_info >= (3, 11)),
    ("Configuration loaded", True),  # We got here, so it loaded
    ("Telemetry initialized", is_telemetry_enabled()),
    ("Azure OpenAI configured", settings.is_azure_configured),
    ("Observability configured", settings.is_observability_configured),
]

for name, passed in checks:
    status = "✅" if passed else "⚠️"
    print(f"  {status} {name}")

print()

required_passed = all([
    sys.version_info >= (3, 11),
    is_telemetry_enabled(),
])

if required_passed:
    print("✅ Your environment is ready for the workshop!")
    if not settings.is_azure_configured:
        print("\n⚠️ Note: Azure OpenAI is not configured.")
        print("   Some notebooks will use mock responses.")
        print("   Configure AZURE_OPENAI_ENDPOINT for full functionality.")
else:
    print("❌ Please fix the issues above before proceeding.")

print()
print("Next: Open notebook 01_basic_agent.ipynb to begin!")